In [1]:
import numpy as np
import pandas as pd
import joblib
from pyswarms.single import GlobalBestPSO


In [2]:
# Load trained models
rf_gain = joblib.load(r"D:\UST Project\UST_Analog_automation\models\Opam\rf_gain_forward.pkl")
rf_ugf  = joblib.load(r"D:\UST Project\UST_Analog_automation\models\Opam\rf_ugf_forward.pkl")
rf_pm   = joblib.load(r"D:\UST Project\UST_Analog_automation\models\Opam\rf_pm_forward.pkl")


In [3]:
df = pd.read_csv(r"D:\UST Project\UST_Analog_automation\data\processed\Opam\pm_ml_ready.csv")
df.columns = df.columns.str.strip().str.lower()

W_MIN, W_MAX = df["w"].min(), df["w"].max()
X_MIN, X_MAX = df["x"].min(), df["x"].max()
Y_MIN, Y_MAX = df["y"].min(), df["y"].max()

bounds = (
    [W_MIN, X_MIN, Y_MIN],
    [W_MAX, X_MAX, Y_MAX]
)

print("Bounds locked:")
print(bounds)


Bounds locked:
([np.float64(3.84e-05), np.float64(2.4e-06), np.float64(4.8e-06)], [np.float64(8.96e-05), np.float64(5.6e-06), np.float64(1.12e-05)])


In [4]:
GAIN_TARGET = 60.0     # dB
UGF_TARGET  = 200.0    # MHz
PM_TARGET   = 60.0     # deg


In [ ]:
GAIN_SCALE = 1.0          # dB
UGF_SCALE  = UGF_TARGET   # normalize by target
PM_SCALE   = 10.0         # deg

def norm_err(pred, target, scale, cap=10.0):
    err = np.abs(pred - target) / scale
    return np.minimum(err, cap)


def fitness(X):
    """
    X shape: (n_particles, 3)
    Returns: (n_particles,)
    """

    W = X[:, 0]
    Xv = X[:, 1]
    Y = X[:, 2]

    params = pd.DataFrame(
    np.column_stack([W, Xv, Y]),
    columns=["W", "X", "Y"]
    )

    gain_pred = rf_gain.predict(params)
    ugf_pred  = rf_ugf.predict(params)
    pm_pred   = rf_pm.predict(params)

    gain_err = norm_err(gain_pred, GAIN_TARGET, scale=1.0)
    ugf_err  = norm_err(ugf_pred,  UGF_TARGET,  scale=UGF_TARGET)
    pm_err   = norm_err(pm_pred,   PM_TARGET,   scale=10.0)

    return gain_err + ugf_err + pm_err


In [13]:
print(rf_gain.feature_names_in_)


['W' 'X' 'Y']


In [14]:
optimizer = GlobalBestPSO(
    n_particles=40,
    dimensions=3,
    options={
        "c1": 1.5,
        "c2": 1.5,
        "w": 0.7
    },
    bounds=bounds
)

best_cost, best_pos = optimizer.optimize(
    fitness,
    iters=120
)


2026-01-21 17:30:28,345 - pyswarms.single.global_best - INFO - Optimize for 120 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}
pyswarms.single.global_best:   0%|          |0/120

pyswarms.single.global_best: 100%|██████████|120/120, best_cost=4.58
2026-01-21 17:31:17,133 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 4.577503734087055, best pos: [4.53444219e-05 2.41610289e-06 9.52021432e-06]


In [21]:
print("\n===== Inverse Design Result =====")

if best_cost > 5.0:
    print("❌ Specs are likely INFEASIBLE within given bounds.")
    print("Best cost:", best_cost)
else:
    W, X, Y = best_pos

    print("✅ Optimal parameters found")
    print(f"W = {W:.6e}")
    print(f"X = {X:.6e}")
    print(f"Y = {Y:.6e}")

    params = pd.DataFrame(
            np.column_stack([W, X, Y]),
            columns=['W', 'X', 'Y']
            )

    print("\nPredicted performance:")
    print("Gain:", rf_gain.predict(params)[0], "dB")
    print("UGF :", rf_ugf.predict(params)[0],  "MHz")
    print("PM  :", rf_pm.predict(params)[0],   "deg")

    print("\nFinal cost:", best_cost)



===== Inverse Design Result =====
✅ Optimal parameters found
W = 4.534442e-05
X = 2.416103e-06
Y = 9.520214e-06

Predicted performance:
Gain: 56.16436503976338 dB
UGF : 130.3202349066604 MHz
PM  : 63.934699483837385 deg

Final cost: 4.577503734087055
